In [1]:
import pandas as pd

In [2]:
def calculate_final_driver_places(conn: str) -> pd.DataFrame:
    """
    Calculate the final positions of drivers based on their points in the last race of each year.

    :param conn: The database connection.
    :return: DataFrame containing year, driverId, driver_nationality, driver_points, and driver_end_position columns.
    """

    points_query = '''
        SELECT r.year, ds.driverId, d.nationality AS driver_nationality, ds.points AS driver_points
        FROM driver_standings ds
        INNER JOIN (
            SELECT r1.year, MAX(r1.date) AS last_race_date
            FROM races r1
            GROUP BY r1.year
        ) last_race ON ds.raceId = (
            SELECT r2.raceId
            FROM races r2
            WHERE r2.year = last_race.year AND r2.date = last_race.last_race_date
            LIMIT 1
        )
        INNER JOIN races r ON ds.raceId = r.raceId AND r.date = last_race.last_race_date
        INNER JOIN drivers d ON ds.driverId = d.driverId;
    '''

    points_df = pd.read_sql_query(points_query, conn)

    points_df = points_df.sort_values(by=['year', 'driver_points'], ascending=[True, False])

    points_df['driver_end_position'] = points_df.groupby('year')['driver_points'].rank(ascending=False, method='min').astype(int)

    return points_df
